In [1]:
# logging and decorators
import logging as log

# tensorflow modules
import tensorflow as tf
import tensorflow_text as tf_text
from tensorflow.keras import layers

# necessary for visualization and user input
import ipywidgets as widgets
from ipywidgets import interact_manual, interactive, interact, VBox, HTML
from IPython.display import display, clear_output

# modules from backend
from interactive_inference_backend import ModelLoader, StoryTokenizer, 
from interactive_inference_backend import reserved_tokens, vocab_path

In [23]:
model = ModelLoader(StoryTokenizer(reserved_tokens, vocab_path),
                            d_model=512,
                            n_stacks=2,
                            h_att=4,
                            load_model=True,
                            model_load_path="model_N2_h4_d512_t20230831-134344")

# Interaktive Erklärung der Transformerarchitektur
## Einleitung
### Kurzübersicht Transformer

Transformer-Modelle sind eine von Vaswani et al. [^vaswani2017] vorgeschlagene Architektur für das Modellieren von sequentiellen Daten. Im Gegensatz zu den vorher genutzten Architekturen wie RNNs [Cite RNN] oder CNNs [Cite CNN] ermöglichen Transformer allerdings das parallele Verarbeiten der sequentiellen Daten während des Trainings und ermöglichen dadurch mit erheblich reduzierter Trainingszeit und -rechenkapazität sequentielle Daten zu verarbeiten [Cite comparisson].

Um diese parallele Verarbeitung zu erreichen nutzen Transformermodelle Aufmerksamkeitsblöcke. Aufmerksamkeitsblöcke sind trainierbare Matrizen, die von einem Eingabevektor auf einen Ausgabevektor projizieren. Also eine sequentielle Datenstruktur in Vektorform in eine ebenfalls sequentielle Datenstruktur in Vektorform verarbeiten, indem eine Matrixmultiplikation auf dein Eingabevektor angewandt wird. Wie das funktioniert wird in [#Aufmerksamkeitsmechanismus] gezeigt.
Diese Matrizen projizieren also im Idealfall von jedem Eintrag im Eingabevektor genau diejenige Information auf einen Eintrag im Ausgabevektor, die an der jeweiligen Stelle die Ausgabe beeinflussen soll. Da hierbei die Ausgabe i nicht von der Ausgabe i-1 abhängt, können wir parallel die komplette Ausgabe zu jeder Eingabe erzeugen und mit einem Gradient-Descent [Cite] Verfahren unser Modell trainieren
Da wir während der Inferenz (Erklärung was Inferenz ist) von sequentiellen Daten aber meist nur die vorhergehende Daten nutzen möchten benutzen Transformer Architekturen eine Maske, die nach der Anwendung der Matrixmultiplikation alle Informationen, die nachfolgende Daten liefern herausfiltern, sodass schon während dem Training nur die Verbindung zwischen der aktuellen Position und den vorhergehenden gelernt wird.
Durch wiederholen von Aufmerksamkeitsblöcken und einem nachfolgenden Feed-Forward Netzwerk entsteht so eine Architektur zur Verarbeitung sequenzieller Daten, die verglichen mit vorherigen Architekturen weniger Rechenkapazität benötigt und trotzdem eine wesentlich reduzierte Trainigszeit aufweist. [!Zitat]

<span style="color:red"> Hier eine grafische Darstellung von Eingabe- zu Ausgabevektor darstellen. Wie werden sequentielle Daten verarbeitet und wie macht das ein Transformer </span>

### Ziel dieses interaktiven Artikels

Ziel dieses interaktiven Artikels soll es sein die Architektur die [Vaswani2017] beschreiben in ihren einzelnen Komponenten darzustellen. Der Fokus liegt hierbei darauf die Prozesse, die während der Verarbeitung sequentieller Daten stattfinden, grafisch durch interaktive Anwendungen darzustellen der Nutzer den Einfluss unterschiedlicher Architekturbausteine auf verschiedene Eingaben anschaulich klar wird. Ziel ist es, die verschiedenen Bausteine so zu erklären, dass einem möglichen Anwender die Implementation einer Tranformer-Architektur, durch ein Verständnis des Nutzens der einzelnen Architekturbausteine, erleichtert wird.

Der Artikel soll dazu dienen eine Implementierung ohne einschlägiges Vorwissen, z.B. im Kontext von KMUs die sich bisher noch nicht mit der Modellierung sequentieller Daten beschäftigt haben, zu ermöglichen und Programmierer in der Designentscheidungen in der Transformerarchitektur zu unterstützen.
[Vaswani2017] und viele der auf ihrer Architektur aufbauenden wissenschaftlichen Arbeiten(Zitate), Erklärartikel oder -video (Zitate) beschränken sich darauf den Aufmerksamkeitsmechanismus ausführlich darzustellen. Dabei werden die Designentscheidungen für trainingsrelevante Elemente wie Dropout (Zitat), Residuale Verbindungen (Zitat) sowie die Beschreibung bereits etablierter Methoden wie Byte-Pair Encoding (zitat) als Einbettung oder Log-Softmax (Zitat) vernachlässigt. Diese Elemente sollen hier aber ebenfalls dargestellt werden.

### Architekturübersicht

In Abbildung 1 ist eine vollständige Darstellung aller Architekturelemente zu finden, die Teil der Transformerarchitektur sind. Dabei unterscheiden wir vier verschiedene Elemente. Prozesse und Methoden werden in Schwarz dargestellt. Die in diesen Prozessen generierte Daten werden in Blau dargestellt. Trainierbare Parameter werden in Gelb dargestellt. Hyperparameter, also festzulegende Eingabeparameter für das Modell werde in Grau dargestellt. Ebenfalls in Grau werden Eingabe- und Ausgabedaten dargestellt.
Unsere Abbildung ist insofern komplett, als sie jeglichen Weg zeigen, den Daten durch das Modell nehmen können. Eine Darstellung in der Form eines Encode-Decoder Netzwerks [Encoder-Decoder], wie sie [Vaswani2017] und andere (Zitat) nutzen findet sich in Abbildung 2.


##### Abbildung 1
<a href="#fig:input">Hier</a> sehen sie die komplette Darstellung der Architektur eines Transformermodells.

<figure id="fig:input" style="height: 700px;">
  <img src="./img/tf_arch_full.jpg" style="height: 1400px;" alt="Eingabepipeline mit Tokenizer, Embedding und Positional Encoding."/>
  <figcaption>Abbildung 1: Transformerarchitektur</figcaption>
</figure>
<br>


#### Encoder-Decoder

Eine Encoder-Decoder Architektur besteht aus zwei voneinander getrennten neuronalen Netzen dem Encoder und dem Decoder, die aber zusammen trainiert werden. Diese Architektur wurde von [Cho 2014 Learning Phrase Representations using RNN Encoder–Decoder for Statistical Machine Translation] vorgeschlagen und dann vor allem im Kontext von Neural Machine Translation (NMT) verwendet. Der Vorteil von Encoder-Decoder liegt darin, dass man sowohl den Encoder als auch den Decoder ersetzen kann. Im Idealfall könnte man im Bereich von NMT pro Sprache einen Encoder und einen Decoder trainieren, um dann beliebige Übersetzungen zwischen allen Sprachen zu generieren. Da auch [NEURAL MACHINE TRANSLATION BY JOINTLY LEARNING TO ALIGN AND TRANSLATE], die zuerst Aufmerksamkeitsmechanismen für Rekurrente Neuronale Netze (RNN) vorgeschlagen haben, eine Encoder-Decoder Architektur verwenden, verwundert es nicht, dass auch [Attention Is All You Need] eine solche Architektur wählten. Insbesondere da [Attention is All You Need] Transformer ebenfalls für NMT benutzen.
Transformer zur Textvervollständigung können auch ausschließlich auf Decodern beruhen. In unserem Beispiel wird der Encoder zum enkodieren der Eingabe verwendet, um dann im Decoders die Eingabe als Anfang der Ausgabe zu verwenden. In Abbildung 2 sieht man wie Encoder und Decoder zusammenarbeiten. Der Decoder benutzt Selbstaufmerksamkeit, um seinen Input zu verarbeiten, nutzt dann <span style="color:red">Source-Attention</span>, für den nächsten Schritt, um final erneut Selbstaufmerksamkeit zu benutzen, wohingegen der Encoder ausschließlich über Selbstaufmerksamkeit funktioniert. Näheres siehe im Kapitel Attention.

##### Abbildung 2

<span style="color:red">
Darstellung des Encoder-Decoder Prinzips
Darstellung der Layer im Encoder und Decoder
</span>
Hier können Sie nun einen Beispielsatz zuerst vom Encoder kodieren lassen, um ihn dann im nächsten Schritt vom Decoder dekodieren zu lassen und damit eine Ausgabe zu erhalten.
<span style="color:red">Interaktive Anwendung Encoder dann Decoder</span>


In [25]:
tokenizer = StoryTokenizer(reserved_tokens, vocab_path)
transformer = model.model

input_widget_enc_dec = widgets.Text(
    value='Encoder-Decoder Test',
    description='Ihre Eingabe:',
    continuous_update=False,  # updates value only when you finish typing or hit "Enter"
    layout = widgets.Layout(width='auto', margin='0px 0px 10px 0px')
)

button_widget_enc = widgets.Button(description='Enkodiere die Eingabe',
                               layout = widgets.Layout(width='auto'))
button_widget_dec = widgets.Button(description='Dekodiere die Eingabe',
                               layout = widgets.Layout(width='auto'))

output_widget_enc = widgets.Output(layout = widgets.Layout(width='auto'))
output_widget_dec = widgets.Output(layout = widgets.Layout(width='auto'))

def on_button_click_enc(b):
  with output_widget_enc:
    output_widget_enc.clear_output()  # clear the previous output
    #output_widget_dec.clear_output()

    # Convert input to tensor if it is not already
    # Create a dynamic tensor to store output
    # Make sure tensor_input is 2-D
    tensor_input = tf.convert_to_tensor(input_widget_enc_dec.value)
    if len(tensor_input.shape) == 0:
      tensor_input = tensor_input[tf.newaxis]
    # tokenize and encode input
    # Identify end token of the input
    tokenized_input = tokenizer.tokenize(tensor_input).to_tensor()
    input_without_eos = tokenized_input[:, :-1]
    context = transformer.encode(input_without_eos, None)

    VisualWrapper.display_text('Beispieltext')
    VisualWrapper.color_bar(context)

def on_button_click_dec(b):
  with output_widget_dec:
    output_widget_dec.clear_output()

    # Convert input to tensor if it is not already
    # Create a dynamic tensor to store output
    # Make sure tensor_input is 2-D
    tensor_input = tf.convert_to_tensor(input_widget_enc_dec.value)
    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    if len(tensor_input.shape) == 0:
      tensor_input = tensor_input[tf.newaxis]
    # tokenize and encode input
    # Identify end token of the input
    tokenized_input = tokenizer.tokenize(tensor_input).to_tensor()
    input_without_eos = tokenized_input[:, :-1]
    context = transformer.encode(input_without_eos, None)

     # Write the input tokens (excluding the last one) to the output array
    for i, value in enumerate(tokenized_input[0][:-1]):
      output_array = output_array.write(i, value)

    dec_input = output_array.concat()[tf.newaxis]

    dec_out = transformer.decode(context, None, dec_input, None)

    VisualWrapper.display_text('Beispieltext')
    VisualWrapper.color_bar(dec_out)

button_widget_enc.on_click(on_button_click_enc)
button_widget_dec.on_click(on_button_click_dec)

widgets.VBox([input_widget_enc_dec, widgets.HBox([widgets.VBox([button_widget_enc, output_widget_enc]), widgets.VBox([button_widget_dec, output_widget_dec])])])
#display(input_widget_enc_dec, button_widget_enc, output_widget_enc, button_widget_dec, output_widget_dec)


#print('tok_out', tokenized_input)
#print('enc_out', context)
#print("dec_out", dec_out)



#### Architekturblöcke

Prinzipiell lassen sich Transformer in mehrere Blöcke einteilen.

1. Eingabeblock

Die Eingabepipeline verarbeitet die Eingabedaten in eine Form die für die Matrixtransformation im Aufmerksamkeitsblock genutzt werden kann. Diese unterscheidet sich für verschiedene Datentypen. In unserem Beispiel nutzen wir Textdaten, die wir durch Tokenisierung und ein Embedding in Tensoren verwandeln.

2. Aufmerksamkeitsblock

Der Aufmerksamkeitsblock verarbeitet Daten in Tensorform und liefert somit eine Abbildung von den Eingabetensoren auf die Ausgabetensoren, die jeweils von den Eingabe- und Ausgabeblöcken interpretiert wird.
Man kann verschiedene Modulvarianten innerhalb eines Aufmerksamkeitsblocks unterscheiden. Aufmerksamkeitsmodule erhalten als Eingabedaten immer drei Tensoren. Diese werden Query, Key und Value genannt. Aus diesen berechnet ein Aufmerksamkeitsmodul eine Ausgabe.
Typischerweise werden Aufmerksamkeitsmodule dabei unterschieden aus welcher Quelle Query, Key und Value stammen. Es gibt Self-Attention bei der Query, Key und Target alle aus einer Quelle stammen, Source-Attention (Hier ist der Query aus einer anderen Quelle als Key und Value, z.B. wenn der Query die Ausgabe eines Encoders ist, während Target die Ausgabe eines Decoderblocks ist).
Desweiteren kann man nach Art des Maskings unterscheiden. Dieses geschieht um  das Aufmerksamkeitsmodul daran zu hindern aus einem Teil der Ausgabedaten zu lernen (z.B. möchte man bei sequentieller Datenverarbeitung verhindern, dass ein Decoderaufmerksamkeitsblock für die Vorhersage der Position i Daten aus den Positionen i+j nutzt.)

3. Ausgabeblock

Die Ausgabepipeline interpretiert die Ausgaben des Aufmerksamkeitsmoduls, sodass sie in eine für menschlichen Gebrauch nützlichen Form vorliegen (typischerweise z.B. Textdaten, Bilddaten, etc.)

In Abbildung 1 entspricht das Aufmerksamkeitsmodul den Prozessen innerhalb der grauen Umrandung, während die Eingabe- und Ausgabepipeline darunter bzw. darüber zu finden sind.

## Input

Der erste Teil eines Transformermodells besteht aus der Eingabepipeline. Diese verarbeitet die Eingabe in das Modells, z.B. die Texteingabe eines Nutzers und bereitet sie darauf vor durch wiederholte Anwendung der Aufmerksamkeitsmodule in einem Transformermodell verarbeitet zu werden. Die Aufmerksamkeitsmodule arbeiten über eine Aufmerksamkeitsmatrix, die aus der jeweiligen Eingabe eine Ausgabe berechnet. Wir müssen also aus einer Eingabe in Textform eine Vektorrepräsentation erzeugen, die alle notwendigen Informationen für ein Modell enthalten, um nützliche Vorhersagen über eine Übersetzung oder eine Textvervollständigung machen zu können.
Hier sehen Sie nochmal den Ausschnitt aus der obigen Grafik, der die Eingabepipeline darstellt.



##### Abbildung 3
<a href="#fig:input">Hier</a> sehen sie nochmal den Ausschnitt aus der obigen Grafik, der die Eingabepipeline darstellt.

<figure id="fig:input" style="height: 700px;">
  <img src="./img/tf_input_pipeline.jpg" style="height: 700px;" alt="Eingabepipeline mit Tokenizer, Embedding und Positional Encoding."/>
  <figcaption>Abbildung 2: Eingabepipeline eines Transformer-Netzwerks</figcaption>
</figure>
<br>



Wie zu erkennen ist, werden während des Training eines Transformermodells ausschließlich die Gewichte für die Einbettung in einen der Modellgröße entsprechenden Vektor mittrainiert.
Prinzipiell besteht die Eingabepipeline aus drei Modulen:

1. Tokenisierung,
2. Einbettung,
3. Positionelle Kodierung.

Dies entspricht den drei Schritten:

1. Umwandlung von Text in eine Zahlenkodierung desselben Textes,
2. Kodierung dieser
<span style="color:red">fortführen!</span>

### Tokenisierung

Bei der Tokenisierung wird der Satz in Textform z.B. "Das ist ein Testsatz." in einen Zahlencode verwandelt. Hierfür kommen verschieden Methoden in Frage. Einer der simpelsten Methoden ist es z.B. jedem Buchstaben eine Zahl zuzuordnen. Das führt allerdings zu einer sehr langen Kodierung. Die entscheidenden Faktoren für eine gute Kodierung sind Vollständigkeit der Kodierung, Länge des kodierten Vektors und Größe des dafür nötigen Vokabulars.
Die Kodierung mit einzelnen Buchstaben ist vollständig (man kann beliebige Zeichenkombinationen kodieren) und besitzt ein kurzes Vokabular (26 für alle Buchstaben plus alle Punktierungs und Sonderzeichen, die im Text vorkommen), allerdings ist die Länge der kodierten Vektoren groß. Andererseits könnte man ein Vokabular an Wörtern nehmen, die führt zu einer viel kürzeren Kodierung, allerdings besteht die Gefahr der Unvollständigkeit und für jedes Wort muss zur Kodierung in einem sehr großem Vokabular nachgeschlagen werden.
Aktuelle Implementationen verwenden Optionen wie das Byte-Pair Encoding. {Cite Sennrich et al. 2016 and Gage 1994}.


Den Tokenizer findet man in unserer <a href="#fig:transformer">Abbildung 1</a> ganz unten und ist der erste Schritt, um eine Eingabe zu verarbeiten
<figure id="fig:tokenizer" style="height: 300px;">
  <img src="./img/tf_tokenizer.jpg" style="height: 300px;" alt="Eingabepipeline mit Tokenizer, Embedding und Positional Encoding."/>
  <figcaption>Abbildung 2: Eingabepipeline eines Transformer-Netzwerks</figcaption>
</figure>
<br>


### Byte-Pair Encoding

Byte-Pair Encoding nutzt ein Vokabular mit einer festgelegten Länge. In unserer Implementation des Tokenizer nutzt er ein Vokabular von 8000 Einheiten. Das Vokabular wird folgendermaßen erstellt:

  1. Ein Text, der für die Erstellung des Vokabulars verwendet wird, wird in eine Sequence von Buchstaben und Symbole zerlegt. Wortenden werden mit einem zusätzlichen Symbol kodiert.
  2. Alle Buchstaben und Symbole werden in das Vokabular aufgenommen.
  3. Nun wird das häufigste 2-Gramm, also zwei aufeinander folgende Symbole, gesucht, das im Text zu finden ist.
  4. Dieses wird ins Vokabular aufgenommen und im Text durch ein einzelnes Symbol ersetzt.
  5. Dieser Prozess wird nun wiederholt, bis die vorgegebene Länge des Vokabulars erreicht ist.


In [26]:
tokenizer = StoryTokenizer(reserved_tokens, vocab_path)

input_widget_tok = widgets.Text(
    value='Tokenizer test',
    description='Your input:',
    continuous_update=False,  # updates value only when you finish typing or hit "Enter"
    layout = widgets.Layout(width='auto', margin='0px 0px 10px 0px')
)

button_widget_tok = widgets.Button(description='Run tokenizer on input',
                               layout = widgets.Layout(width='auto'))

output_widget_tok = widgets.Output(layout = widgets.Layout(width='auto'))

def on_button_click(b):
    with output_widget_tok:
        output_widget_tok.clear_output()  # clear the previous output
        tokens = tokenizer.tokenize(input_widget_tok.value)
        lookup = tokenizer.lookup(tokens)

        VisualWrapper.display_text('Tokens die aus der Eingabe mit Byte-Pair Encoding extrahiert werden:'.rjust(100) + ', '.join([token.decode('utf-8').rjust(10) for token in lookup.numpy()[0]])
                                   .replace(' ', '&nbsp;'))
        VisualWrapper.display_text('Ihre Positionsnummer im Alphabet des Byte-Pair Encoding Algorithmus:'.rjust(100) + ', '.join([str(token).rjust(10) for token in tokens.numpy()[0]])
                                   .replace(' ', '&nbsp;'))

button_widget_tok.on_click(on_button_click)

display(input_widget_tok, button_widget_tok, output_widget_tok)

Text(value='Tokenizer test', continuous_update=False, description='Your input:', layout=Layout(margin='0px 0px…

Button(description='Run tokenizer on input', layout=Layout(width='auto'), style=ButtonStyle())

Output(layout=Layout(width='auto'))

In our test example you can see, how the input string is separated into tokens and then converted into numerical values, depending on the position the token has in our vocabulary.
As you can see our byte-pair encoding vocabulary is extended by an [START] and [END] token, and it also contains elements of type 'abc##' or '##abc'. These represent a sequence at the start or end of a word.



##### Abbildung 4

	<span style="color:red">
  Tokenizer
  </span>

Gage - A New Algorithm for Data Compression.pdf
Sennrich et al. - 2016 - Neural Machine Translation of Rare Words with Subw.pdf
<span style="color:red">
In our test example you can see, how the input string is separated into tokens and then converted into numerical values, depending on the position the token has in our vocabulary.
As you can see our byte-pair encoding vocabulary is extended by an [START] and [END] token, and it also contains elements of type 'abc##' or '##abc'. These represent a sequence at the start or end of a word.
</span>

### Input Embedding

Die Einbettung sorgt dafür, dass die beliebig lange Sequenzen die durch den Tokenizer entsteht in einen Vektor der Modellgröße d_model kodiert werden. Das heißt jedes Token, das zuvor durch eine Zahl kodiert wurde, die der Position entspricht, die das jeweilige Token in einem (in unserer Implementierung 8000 Wörter langen) Tokenwörterbuch zugewiesen bekommen hat, wird nun durch einen Vektor der Länge d_model kodiert. So entsteht ein Tensor der Dimension Anzahl enkodierte Tokens d_model. Die Einbettung ist Teil der vom Modell gelernten Parameter, wie man in der Übersichtsgrafik sehen kann.
Wie man in <a href="#fig:embedding">Abbildung 4</a> sehen kann besitzt die Einbettung trainierbare Gewichte und ist damit Teil der während des Training gelernten Parameter des Modells.



##### Abbildung 5

Wie man in <a href="#fig:embedding">Abbildung 4</a> sehen kann besitzt die Einbettung trainierbare Gewichte und ist damit Teil der während des Training gelernten Parameter des Modells.
<figure id="fig:embedding" style="height: 300px;">
  <img src="./img/tf_embedding.jpg" style="height: 300px;" alt="Eingabepipeline mit Tokenizer, Embedding und Positional Encoding."/>
  <figcaption>Abbildung 4: Eingabepipeline eines Transformer-Netzwerks</figcaption>
</figure>
<br>


In [27]:
class EmbeddingExample():

    def __init__(self) -> None:
        self.tokenizer = StoryTokenizer(reserved_tokens, vocab_path)

        self.input_widget = widgets.Text(
            value = 'Einbettung Test',
            description = 'Ihre Eingabe:',
            continuous_update=False,  # updates value only when you finish typing or hit "Enter"
            layout = widgets.Layout(width='auto', margin='0px 0px 10px 0px')
        )

        self.button_widget = widgets.Button(description='Einbettung erstellen',
                                    layout = widgets.Layout(width='auto'))

        self.output_widget = widgets.Output(layout = widgets.Layout(width='auto'))
        self.old_context = None

    def on_button_click(self, b):
        with self.output_widget:
            self.output_widget.clear_output()  # clear the previous output
            VisualWrapper.reset_visualiser()
            tokens = self.tokenizer.tokenize(self.input_widget.value)
            input_without_eos = tokens[tf.newaxis, :, :-1]
            context = model.model.enc_embed(input_without_eos)
            VisualWrapper.display_text('So sieht die Einbettung der Eingabe aus.')
            VisualWrapper.color_bar(context.to_tensor())

            if self.old_context is not None:
                padded_context, padded_old_context = self.pad_tensors(context, self.old_context)

                VisualWrapper.display_text('So unterscheiden sich die alte und die neue Einbettung voneinander.')
                context_diff = padded_context - padded_old_context
                VisualWrapper.color_bar(context_diff)

            self.old_context = context
    
    def pad_tensors(self, ragged_tensor1, ragged_tensor2):
        # Convert ragged tensors to normal tensors, padding with zeros
        tensor1 = ragged_tensor1.to_tensor()
        tensor2 = ragged_tensor2.to_tensor()

        # Calculate the shapes of the tensors
        shape1 = tf.shape(tensor1)
        shape2 = tf.shape(tensor2)

        # Initialize a list for the target shape
        target_shape = []

        # Iterate over the dimensions of the tensors
        for i in range(shape1.shape[0]):
            # Append the maximum size of the dimension to the target shape
            target_shape.append(tf.maximum(shape1[i], shape2[i]))

        # Convert the target shape to a tensor
        target_shape = tf.stack(target_shape)

        # Initialize lists for the paddings of the tensors
        paddings1 = []
        paddings2 = []

        # Iterate over the dimensions of the tensors
        for i in range(shape1.shape[0]):
            # Append the required padding for the dimension to the paddings
            paddings1.append([0, target_shape[i] - shape1[i]])
            paddings2.append([0, target_shape[i] - shape2[i]])

        # Convert the paddings to tensors
        paddings1 = tf.stack(paddings1)
        paddings2 = tf.stack(paddings2)

        # Pad the tensors to the target shape
        tensor1_padded = tf.pad(tensor1, paddings1)
        tensor2_padded = tf.pad(tensor2, paddings2)

        return tensor1_padded, tensor2_padded

emb_ex = EmbeddingExample()

VisualWrapper.display_text('Hier kannst du einen Text einbetten lassen. Wenn du die Eingabe veränderst wird außerdem gezeigt, wie sich die Einbettung geändert hat.')

emb_ex.button_widget.on_click(emb_ex.on_button_click)
display(emb_ex.input_widget, emb_ex.button_widget, emb_ex.output_widget)


HTML(value='<p style="font-size:18px; color:blue;">Hier kannst du einen Text einbetten lassen. Wenn du die Ein…

Text(value='Einbettung Test', continuous_update=False, description='Ihre Eingabe:', layout=Layout(margin='0px …

Button(description='Einbettung erstellen', layout=Layout(width='auto'), style=ButtonStyle())

Output(layout=Layout(width='auto'))



### Positionale Kodierung

Da in der Einbettung keine Informationen über die Position der verschiedenen Worte kodiert wird, muss diese zusätzlich kodiert werden. Hierfür benutzt die Transformerarchitektur für jede Position der Einbettung (also jedes enkodierte Wort) eine Sinuskurve mit anderer Frequenz und Phase. Hier ist zu sehen, wie die positionale Kodierung für eine 2048 Vektoren lange und 512 Einträge tiefe Einbettung aussieht.

Wie man erkennen kann, basiert die Kodierung darauf, dass Sinuskurven mit kurzer Frequenz eine Unterscheidung von Positionen ermöglichen, die nahe beieinander liegen, da ihre Werte für benachbarte ganze Zahlen sehr verschiedene Werte liefern. Sinuskurven mit langer Frequenz unterscheiden sich erst, wenn zwei Positionen, bzw. die ganze Zahlen, die sie repräsentieren weit voneinander entfernt liegen, dadurch können auch weiter voneinander entfernt liegende Positionen in Relation gesetzt werden. In die Einbettung werden diese verschiedenen Sinuskurven eingefügt, indem die Frequenz der hinzugefügten Sinuskurve von der Tiefe der Einbettung bestimmt wird. In unserer Grafik oben wird also jeweils eine Zeile der Einbettung an der jeweiligen Position hinzugefügt.


In [28]:
@interact(length=(2,2048,1), depth=(2,512,1))
def print_pos_enc(length, depth):
    VisualWrapper.color_bar(positional_encoding(length, depth))

interactive(children=(IntSlider(value=1025, description='length', max=2048, min=2), IntSlider(value=257, descr…


## Trainingsmethoden

### Dropout

Dropout ist eine Methode, die während des Trainingsprozesses eines neuronalen Netzes genutzt wird, um zu verhindern, dass die gelernte Gewichtung eines Modells in einem der Module des Modells zu sehr auf einen einzelnen Prädikator stützt. Dafür werden zwischen zwei Schritten desselben Modells, die trainierbare Gewichte enthalten eine Dropout-Layer eingefügt. Diese setzt zufällig einige der vom Modell generierten Werte auf einen vordefinierten Wert (meistens 0), um den nachfolgenden Schichten diese Informationen vorzuenthalten. Da diese Operation zufällig erfolgt, müssen die nachfolgenden Teile des Modells auf eine möglichst breite Kombination aus Merkmalen setzen, um seine Vorhersagen zu treffen. Somit kann man verhindern, dass Vorhersagen nur aufgrund eines einzigen Merkmals der vorherigen Ausgabe gemacht werden.
Srivastava et al. - Dropout A Simple Way to Prevent Neural Networks f.pdf

In [29]:
tokenizer_drop = StoryTokenizer(reserved_tokens, vocab_path)

input_widget_drop = widgets.Text(value = 'Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test',
                                 description = 'Ihre Eingabe:',
                                 continuous_update=False,  # updates value only when you finish typing or hit "Enter"
                                 layout = widgets.Layout(width='auto', margin='0px 0px 10px 0px')
                                 )

length_widget_drop = widgets.IntSlider(value=30,
                           min=2,
                           max=2048,
                           description='Länge des Tensors:',
                           continuous_update=False,  # updates value only when you finish typing or hit "Enter"
                           )
depth_widget_drop = widgets.IntSlider(value=512,
                          min=2,
                          max=512,
                          description='Tiefe des Tensors:',
                          continuous_update=False,  # updates value only when you finish typing or hit "Enter"
                          )
dropout_widget = widgets.FloatSlider(value=0.1,
                              min=0,
                              max=0.9,
                              step=0.1,
                              description='Dropoutrate:',
                              continuous_update=False,  # updates value only when you finish typing or hit "Enter"
                              )

def out(length, depth, dropout, input):
    VisualWrapper.reset_visualiser()
    dropout_layer = layers.Dropout(dropout)
    one_tensor = tf.ones([length, depth])
    dropout_tensor = dropout_layer(one_tensor, training=True)
    VisualWrapper.color_bar(dropout_tensor)

    tokens = tokenizer_drop.tokenize(input)
    input_without_eos = tokens[tf.newaxis, :, 1:-1]
    context = model.model.enc_embed(input_without_eos)
    context_drop = dropout_layer(context, training=True)
    VisualWrapper.display_text('Für normale Dropoutwerte zwischen 0 und 0.3 sieht man die Veränderungen an tatsächlichen Vektoren nur schlecht, da viele Werte eines Tensors schon nahe bei 0 liegen.')
    VisualWrapper.color_bar(context.to_tensor())
    VisualWrapper.color_bar(context_drop.to_tensor())
    

output_widget_dropout = widgets.interactive_output(out,
                                                   {'length': length_widget_drop, 'depth': depth_widget_drop, 'dropout': dropout_widget, 'input': input_widget_drop}, 
                                                   )

display(length_widget_drop, depth_widget_drop, dropout_widget, input_widget_drop, output_widget_dropout)


IntSlider(value=30, continuous_update=False, description='Länge des Tensors:', max=2048, min=2)

IntSlider(value=512, continuous_update=False, description='Tiefe des Tensors:', max=512, min=2)

FloatSlider(value=0.1, continuous_update=False, description='Dropoutrate:', max=0.9)

Text(value='Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Test Tes…

Output()



### Normalisierung

Normalisierung ist eine Technik, die von [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift] eingeführt wurde. In tiefen neuronalen Netzen, mit nicht-linearen Aktivierungsfunktionen wie der Sigmoid-Funktion g(x) = 1/(1+exp(-x)) trainiert werden gilt, das g’(x) -> 0 für |x|-> inf. Das führt dazu, dass diese Funktionen in einen Bereich geraten können, in der der Gradient für Stochastic Gradient Descent (SGD) minimal wird und man spricht vom Vanishing Gradient Problem. In tiefen neuronalen Netzen ergibt sich hierbei das Problem, dass eine Layer z = g(Wx + b) mit der Sigmoid-Funktion g versucht den Output des gesamten vorherigen Netzes x zu gewichten. Dabei hängen sowohl W als auch b von x ab. Da sich während des Trainings alle Layers des Netzes fortwährend aktualisieren, ändert sich auch der Input x fortwährend. Die Tiefe eines neuronalen Netzes erhöht die Wahrscheinlichkeit für einen Vanishing Gradient. Dieser Effekt wird von [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift] Internal Covariate Shift genannt. 

Ba et al. - 2016 - Layer Normalization.pdf
1502.03167.pdf
<span style="color:red">Interactive Application - In- Output of LayerNorm Comparison</span>



In [30]:
VisualWrapper.reset_visualiser()
tokenizer_norm = StoryTokenizer(reserved_tokens, vocab_path)

input="Test"

tensor_input = tf.convert_to_tensor(input)
if len(tensor_input.shape) == 0:
    tensor_input = tensor_input[tf.newaxis]
# tokenize and encode input
# Identify end token of the input
tokenized_input = tokenizer_norm.tokenize(tensor_input).to_tensor()
input_without_eos = tokenized_input[:, :-1]
context = model.model.encode(input_without_eos, None)

VisualWrapper.visualize_data(id='layer')
# Hier gibt es ein Problem, weil man das Modell nicht ohne die Layernorm laufen lassen kann. Hierfür muss erst eine Lösung implementiert werden.




### Skalierung mit sqr(d_k)

Dasselbe Problem des Vanishing Gradient könnte innerhalb der Aufmerksamkeitsfunktion auftauchen, da hier softmax(QK^T)V berechnet wird, das Skalarprodukt QK^T mit d_model skaliert und die Softmaxfunktion
    
    sigma(z)_i = exp(z_i)/sum_j=1^N(exp(z_j)) für j=1,...,N

somit leicht in einen saturierten Bereich mit extrem kleinen Gradienten gerät. Deshalb wird QK^T mit sqr(d_model) skaliert: softmax(QK^T/sqr(d_model)), um die Skalierung mit d_model zu minimieren.

### Residuale Verbindung

Die Idee für das Nutzen von Residualen Verbindungen kommt von [He et al. - 2015 - Deep Residual Learning for Image Recognition]. Die Autoren stellten fest, dass bei tiefen neuronalen Netzen (Tiefe meint hier die Anzahl an Schichten des neuronalen Netzes) sowohl die Genauigkeit während des Trainings als auch die Genauigkeit auf dem Testdatensatz mit zunehmender Tiefe schlechter wird.
Da durch Normalisierung bereits sichergestellt ist, dass schwindende oder explodierende Gradienten kein Problem darstellen, scheitert die Optimierung der neuronalen Netze aus anderen Gründen.
Einer der Gründe hierfür liegt vermutlich darin, dass die tieferen Schichten eines Modells zu Beginn des Trainings sehr viel stärker zur Ausgabe beitragen, als die vorhergehenden Schichten. Sie werden somit zuerst trainiert. Die weniger tiefen Schichten werden erst ausreichend trainiert, wenn in den tiefen Schichten keine Optimierung mehr möglich ist.
Residuale Verbindungen ersetzen eine Schicht F(x) durch ihre residuale Verbindung H(x) = F(x)+x. In das Ergebnis von H(x) geht also sowohl der Output, als auch der Input von F direkt mit ein. Wendet man dieses Prinzip auf die Schichten tiefer neuronaler Netze an, sorgt das dafür, dass gleich zu Beginn, der Output der wenig tiefen Schichten relevant in den Output des gesamten Netzes einfließt, denn es gilt für das gesamte Netz N:
N(x) = H_n(H_n-1(x)) + H_n-1(x) = H_n(H_n-1(x)) + H_n-1(H_n-2(x)) + … + H_2(H_1(x)) + H_1(x) 
Wie man in Abbildung 1 sehen kann, haben alle Aufmerksamkeitsmodule, sowie alle Feed-Forward Schichten eine residuale Verbindung.
Hier können Sie ausprobieren, wie groß die Veränderung ist, die eine residuale Verbindung einer Feed-Forward Layer aus unserem Transformermodell hinzufügt:

<span style="color:red">Interaktiver Vergleich mit/ohne residuale Verbindung</span>


In [31]:
# Hier soll ein interaktiver Vergleich zwischen einer Schicht ohne/mit residualen Verbindungen stehen. Das geht allerdings nur, wenn man diese deaktivieren kann. Das muss implementiert werden.



## Modellschichten

### Aufmerksamkeit

Die Neuerung von Transformern im Vergleich zu vorangegangenen Lösungen für NMT ist es, allein auf den Mechanismus als Architektur für das Verarbeiten von Sprache zu setzen. Aufmerksamkeit wird auch schon von [NEURAL MACHINE TRANSLATION BY JOINTLY LEARNING TO ALIGN AND TRANSLATE] zur Verbesserung von RNNs zur Übersetzung von Texten verwendet.
Der Aufmerksamkeitsmechanismus, den [Attention is all you need] beschreiben orientiert sich von der Idee dabei an einer Suchanfrage. [Buch zitieren] Die Aufmerksamkeitsschicht bekommt dabei zwei oder eigentlich drei Eingaben: den Query (Q), den Key (K) und den Value (V). In der Praxis erhalten aber Key und Value in Transformern immer dieselbe Eingabe und häufig sind Query, Key und Value sogar identisch. Aus welcher Quelle Query, Key und Value kommen unterscheidet unterschiedliche Formen von Aufmerksamkeit. So nennen wir Selbstaufmerksamkeit denjenigen Fall indem Q=K=V gilt und Kreuzaufmerksamkeit denjenigen Fall indem der Query aus der Ausgabe des Encoder besteht und Key und Value beide aus der Ausgabe eines Decoderblocks.

Um zu erklären, wie Aufmerksamkeit funktioniert, sollten wir aber zunächst davon ausgehen, dass Query-, Key- und Value-Eingabe verschieden sind. Ich schreibe bewusst von der Eingabe, da in jeder Aufmerksamkeitsschicht zunächst Query-, Key- und Value-Eingabe Q_in, K_in, V_in mit Hilfe von gewichteten Matrix W^Q, W^K und W^V in Query, Key und Value Q= Q_in W^Q, K = K_in W^K, V = V_in W^V umgewandelt werden. Diese gewichteten Matrizen W^Q, W^K, W^V sind die trainierbaren Gewichte einer Aufmerksamkeitsschicht. In ihnen wird das Ergebnis der Aufmerksamkeitsschicht festgelegt, da alle nachfolgenden Prozesse deterministisch sind.
Betrachten wir aber, was passiert, wenn Query, Key und Value durch diese Matrizen gewichtet werden.

#### Aufmerksamkeitsfunktion

Die Aufmerksamkeitsfunktion hat die Eingaben Q, K, V und lautet:

	Attention(Q, K, V) = softmax(QK^T/sqr(d_k)) V

Es wird also zuerst das Kreuzprodukt aus Q und K gebildet. Dieses Produkt wird skaliert, wie in Skalierung mit sqr(d_k) zu lesen ist, und dann die Softmax-Funktion

	sigma(x)_i = exp(x_i)/sum_j=1^n(exp(x_j)) für i=1,...,n
positionsweise berechnet. Nennen wir 

	softmax(QK^T/sqr(d_k)) = Score(Q,K),
dann ist 

	Attention(Q, K, V) = Score(Q,K)*V 
eine Funktion, die V mit einem Vektor multipliziert, wobei |Score(Q,K)| = 1.

<span style="color:red">Hier gilt vermutlich, das softmax komponentenweise angewandt wird, was aber überprüft werden sollte, bevor wir es so aufnehmen. </span>
Score(Q,K) gibt also an, mit welchem Anteil jeder Eintrag von V in den Ausgabevektor Attention(Q,K,V) eingehen soll. Eine gute grafische Erklärung dieser Methode findet man in [https://jalammar.github.io/illustrated-transformer/].
Da mit Score(Q,K) nun eine Gewichtung besteht, wie stark die Ausgabe Attention(Q,K,V)_i von V_j abhängt, kann man diese als Aufmerksamkeitsgewichtung in Matrixform sehr gut darstellen.


In [32]:
tokenizer_attn = StoryTokenizer(reserved_tokens, vocab_path)
attn_model = WordComplete(StoryTokenizer(reserved_tokens, vocab_path), model.model, max_length=32)

input_widget_attn = widgets.Text(
    value='A longer test sentence is more interesting, as it allows to see dependencies more clearly.',
    description='Ihre Eingabe:',
    continuous_update=False,  # updates value only when you finish typing or hit "Enter"
    layout = widgets.Layout(width='auto', margin='0px 0px 10px 0px')
)

button_widget_attn = widgets.Button(description='Aufmerksamkeit berechnen',
                               layout = widgets.Layout(width='auto'))

output_widget_attn = widgets.Output(layout = widgets.Layout(width='auto'))

def on_button_click(b):
    with output_widget_attn:
        #VisualWrapper.n_vis_layers_per_class['MultiHeadedAttention'] = 6
        #output_widget_attn.clear_output()  # clear the previous output

        # Convert input to tensor if it is not already
        # Create a dynamic tensor to store output
        # Make sure tensor_input is 2-D
        tensor_input = tf.convert_to_tensor(input_widget_attn.value)
        output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
        if len(tensor_input.shape) == 0:
            tensor_input = tensor_input[tf.newaxis]
        # tokenize and encode input
        # Identify end token of the input
        tokenized_input = tokenizer.tokenize(tensor_input).to_tensor()
        input_without_eos = tokenized_input[:, :-1]
        context = transformer.encode(input_without_eos, None)

        # Write the input tokens (excluding the last one) to the output array
        for i, value in enumerate(tokenized_input[0][:-1]):
            output_array = output_array.write(i, value)

        dec_input = output_array.concat()[tf.newaxis]

        dec_out = transformer.decode(context, None, dec_input, None)

        VisualWrapper.visualize_data(id='attention')
        #VisualWrapper.n_vis_layers_per_class['MultiHeadedAttention'] = 1
            

button_widget_attn.on_click(on_button_click)

display(input_widget_attn, button_widget_attn, output_widget_attn)

Text(value='A longer test sentence is more interesting, as it allows to see dependencies more clearly.', conti…

Button(description='Aufmerksamkeit berechnen', layout=Layout(width='auto'), style=ButtonStyle())

Output(layout=Layout(width='auto'))


#### Multi-headed Aufmerksamkeit

In der Praxis hat sich bewährt parallel mehrere dieser Aufmerksamkeitsmechanismen durchzuführen. Dazu werden zu Beginn h gewichteten Matrizen W_i^Q, W_i^K, W_i^V i= 1,...,h eingeführt. Diese erzeugen also h verschiedene Matrixtripel Q_i, K_i, V_i und somit ergeben sich h verschiedene Ausgaben H_i = Attention(Q_i, K_i, V_i) i=1,...,h.
Diese werden nun zu einer einzigen Ausgabe zusammengeführt, indem wir MultiHeadAttention(Q_in,K_in,V_in) = Concat(H_1,..., H_h)W^O berechnen. Dabei ist Concat(A_1,...,A_n) das hintereinanderschreiben mehrerer Matrizen und W^O eine weitere trainierbare Matrix, die die verschiedenen Ausgaben H_1,..., H,h gewichtet.
Deshalb sehen wir in der obigen Ausgabe auch x verschiedene Matrizen, die Score(Q_i,K_i) darstellen.

#### Verschiedene Aufmerksamkeitstypen

In der Architektur werden verschiedene Aufmerksamkeitstypen unterschieden. Es gibt dabei zwei Variablen die beeinflussen, welche Art von Aufmerksamkeit wir verwenden. Die erste Variable ist woher die Eingaben Q_in, K_in und V_in kommen. Die zweite Variable ist die Maskierung, die wir auf Score(Q,K) anwenden.

##### Selbst-Aufmerksamkeit

Die Aufmerksamkeit nennen wir Selbst-Aufmerksamkeit, wenn gilt Q_in=K_in=V_in. Wenn sich Score(Q,K) also bildlich gesprochen daraus ergibt, welche Aufmerksamkeit jede Position der Eingabe auf eine andere Position derselben Eingabe legt und diese Aufmerksamkeit auf die Eingabe selbst angewandt wird.

##### Kreuz-Aufmerksamkeit

Wie nennen die Aufmerksamkeit Kreuz-Aufmerksamkeit, wenn gilt Q_in |= K_in = V_in. Wenn sich Score(Q,K) also daraus ergibt, welche Aufmerksamkeit jede Position einer Eingabe Q_in auf die Positionen einer zweiten Eingabe K_in gibt und dieser Aufmerksamkeitsscore auf die zweite Eingabe angewandt wird. Dies ist zum Beispiel in der Encoder-Decoder der Fall, wenn Q_in sich aus der Ausgabe des Encoder ergibt und K_in = V_in ein Zwischenergebnis des Decoders ist.

##### Maskierte Aufmerksamkeit

Ein Fall von maskierter Aufmerksamkeit liegt dann vor, wenn bestimmte Werte von Score(Q,K) maskiert werden. Das ist zum Beispiel beim Subsequent Masking der Fall, hier wird Score(Q,K)i,j = -inf gesetzt für alle Eintrage j>i. Dadurch wird verhindert, dass die Ausgabe Attention(Q,K,V)_i sich auf die Werte V_j, j>i stützt. Z.B. wird während des Trainings im Decoder dadurch verhindert, dass das Model lernt Informationen aus den zukünftigen Einträgen V_j, j>i zu benutzen, um V_i vorherzusagen. Man sieht gut in der Darstellung von Score(Q,K), dass die Werte für j>i dadurch meistens nahe bei 0 liegen.

### Feed-Forward

<span style="color:red">Evtl. Zitat zu einer Quelle über Standard FFN einfügen.</span>
Feed-Forward Netzwerken sind die Standard Implementations eines neronalen Netzes, in der die Neuronen einer Schicht mit jedem Neuron der nachfolgenden Schicht verbunden sind. In der hier realisierten Implementation bestehen sie aus einer Inputschicht und einer Outputschicht der Größe d_model sowie einer Hidden-Layer der Größe 2048. Dies ist die Standardimplementation von Transformermodellen.
In [FNet] wird gezeigt, dass man die Aufmerksamkeitslayers vollständig durch die lineare Fast-Fourier Transformation ersetzen kann, was zeigt, dass die Feed-Forward Layers durchaus einen erheblichen Anteil an der Interpretation des Inputs eines Transformermodells haben.

### Maskieren

Das Maskieren des Inputs ist eine wichtige funktionale Komponente der Transformerarchitektur. Beim Maskieren handelt es sich in Wirklichkeit um zwei Mechanismen, die zwar dieselbe Funktionsweise besitzen, aber sehr unterschiedliche Aufgaben in der Architektur besitzen. Einerseits das Subsequent Masking und das Padding Masking. Das Padding Masking stellt jediglich sicher, dass nur Positionen mit Inhalt vom Transformer verarbeitet werden, während das Subsequent Masking dafür sorgt, dass der Decoder des Transformers autoregressiv [Zitat Autoregression] ist. Das bedeutet, bei einer Vorhersage für eine Ausgabe an der Position i, soll das Modell nur Informationen aus den Eingabepositionen 1, …,  i-1 nutzen.



<span style="color:red"> Interaktive Anwendung vgl. eines Tensors vor und nach dem Maskieren<span>



#### Padding Masking

Das Padding Masking ist notwendig, da Transformer sequentielle Daten so verarbeiten, als ob sie eine fixe Länge d_model hätten. Das geschieht deshalb, weil Transformer so lernen können jede Position der Ausgabelänge d_model parallel vorherzusagen. <span style="color:red">Evtl. genauere Ausführung.</span>
Um während des Trainings auch Daten mit einer Länge größer oder kleiner d_model zu nutzen, werden längere Sequenzen abgeschnitten und kürzere mit Nullen aufgefüllt. Diese Nullen müssen dann mit Hilfe von Padding Masking für das Training irrelevant gemacht werden. Das geschieht indem man alle Positionen die eine Null enthalten für das Modell auf -inf setzt, sodass sie beim Gradient Descent nicht berücksichtigt werden. [Zitat zu genaueren Erklärung] <span style="color:red">Überprüfen ob das so stimmt!</span>

#### Subsequent Masking

Das Subsequent Masking benutzt die gleich Technik und setzt bestimmte Einträge innerhalb der Aufmerksamkeitslayers auf den Wert -inf. Subsequent Masking und Padding Masking werden dabei gleichzeitig angewandt.
Einfügen einer mathematischen Beschreibung.


## Output

### Log-Softmax

<span style="color:red">Hier fehlt noch der Inhalt</span>

http://arxiv.org/abs/1608.05859


In [33]:
inference_model = WordComplete(StoryTokenizer(reserved_tokens, vocab_path), model.model, max_length=32)

input_widget_inf = widgets.Text(
    value='Test sentence',
    description='Your input:',
    continuous_update=False,  # updates value only when you finish typing or hit "Enter"
    layout = widgets.Layout(width='auto', margin='0px 0px 10px 0px')
)

button_widget_inf = widgets.Button(description='Run interactive inference',
                               layout = widgets.Layout(width='auto'))

output_widget_inf = widgets.Output(layout = widgets.Layout(width='auto'))

def on_button_click(b):
    with output_widget_inf:
        output_widget_inf.clear_output()  # clear the previous output
        inference_model(input_widget_inf.value, interactive=True) # replace this with your function call
        inference_model.print_results(visualisation=True)

button_widget_inf.on_click(on_button_click)

display(input_widget_inf, button_widget_inf, output_widget_inf)


Text(value='Test sentence', continuous_update=False, description='Your input:', layout=Layout(margin='0px 0px …

Button(description='Run interactive inference', layout=Layout(width='auto'), style=ButtonStyle())

Output(layout=Layout(width='auto'))